# Querying Connectomes with NeuPrint

NeuPrint is a set of tools for performing graph analysis on connectomes using the Neo4j graph database and Cypher graph query language. NeuPrint was riginally written by the [FlyEM team](https://www.janelia.org/project-team/flyem) at Janelia Research Campus.

BossDB's NeuPrint connectomes can also be explored using the web UI at https://neuprint.bossdb.io/. 

In [ ]:
!pip install neuprint-python
!pip install requests
!pip install numpy==1.26.4
!pip install pandas==1.5.3

In [1]:
import requests
import json
from neuprint import Client, queries, NeuronCriteria, set_default_client, merge_neuron_properties
from neuprint.utils import connection_table_to_matrix

To get started, you'll need to obtain a NeuPrint token.
* Navigate to https://neuprint.bossdb.io in a web browser.
* Log in with any Google account. You do not have to request an account from us.
* Click your profile picture in the top right, then click Account.
* Copy the auth token into the TOKEN variable.

In [2]:
TOKEN = ""

To avoid repeating this step, set environment variable NEUPRINT_APPLICATION_CREDENTIALS with this token as its value.

## Dataset Discovery

First we'll request info on all the datasets available in BossDB's NeuPrint.

In [3]:
neuprint_host = "https://neuprint.bossdb.io"
datasets_response = requests.get(f"{neuprint_host}/api/dbmeta/datasets", headers={"Authorization": f"Bearer {TOKEN}"})
datasets_metadata = json.loads(datasets_response.content)

We can list the datasets available like so:

In [4]:
dataset_names = list(datasets_metadata.keys())
dataset_names

['witvliet_dataset1',
 'witvliet_dataset2',
 'witvliet_dataset3',
 'witvliet_dataset4',
 'witvliet_dataset5',
 'witvliet_dataset6',
 'witvliet_dataset8']

And then view the NeuPrint metadata for a given dataset.

In [5]:
witvliet_1 = dataset_names[0]
datasets_metadata[witvliet_1]

{'last-mod': '1970-01-01 00:00:00',
 'uuid': 'None',
 'ROIs': ['tail',
  'witvliet_dataset1',
  'Immature module 1',
  'Immature module 2',
  'head'],
 'superLevelROIs': ['Immature module 1', 'Immature module 2', 'head', 'tail'],
 'info': 'https://bossdb.org/project/witvliet2020',
 'hidden': False}

For a list of other REST endpoints which can be used to access NeuPrint data, see https://neuprint.bossdb.io/help/api.

## Querying a Dataset

To perform a Cypher query against a dataset using Python, we create a Client using the NeuPrint Python package. We specify the dataset we want to query against as part of the Client construction, and can see more metadata as a result.

Docs: https://connectome-neuprint.github.io/neuprint-python/docs/index.html

In [6]:
c = Client(neuprint_host, "witvliet_dataset1", token=TOKEN)
set_default_client(c)
queries.fetch_meta()  # uses the first Client object instantiated

{'dataset': 'witvliet_dataset1',
 'info': 'https://bossdb.org/project/witvliet2020',
 'lastDatabaseEdit': '1970-01-01 00:00:00',
 'latestMutationId': 0,
 'logo': 'https://bossdb-assets.s3.amazonaws.com/projects/witvliet2020.jpg',
 'meshHost': 'None',
 'neuroglancerInfo': {},
 'neuroglancerMeta': [],
 'postHPThreshold': 0,
 'postHighAccuracyThreshold': 0,
 'preHPThreshold': 0,
 'primaryRois': ['Immature module 1', 'Immature module 2', 'head', 'tail'],
 'roiHierarchy': {'children': [{'name': 'Immature module 1'},
   {'name': 'Immature module 2'},
   {'name': 'head'},
   {'name': 'tail'}],
  'name': 'witvliet_dataset1'},
 'roiInfo': {'Immature module 1': {'description': 'Immature module 1',
   'hasROI': False,
   'isPrimary': True,
   'parent': 'witvliet_dataset1',
   'post': 1099,
   'pre': 821,
   'showHierarchy': True},
  'Immature module 2': {'description': 'Immature module 2',
   'hasROI': False,
   'isPrimary': True,
   'parent': 'witvliet_dataset1',
   'post': 177,
   'pre': 456,
 

The queries subpackage has some useful queries built in.

In [7]:
# List all neurons in an ROI
criteria = NeuronCriteria(rois=["tail"])
neuron_df, roi_counts_df = queries.fetch_neurons(criteria)
neuron_df

,bodyId,instance,type,pre,post,downstream,upstream,status,cropped,statusLabel,somaLocation,roiInfo,synweight,inputRois,outputRois
0,71,AVAR,i,0,19,0,19,Traced,False,Traced,"[10552, 10846, 145]","{'Immature module 1': {'post': 19, 'upstream':...",19,"[Immature module 1, head, tail]",[]
1,77,AVAL,i,0,16,0,16,Traced,False,Traced,"[10552, 10846, 145]","{'Immature module 1': {'post': 16, 'upstream':...",16,"[Immature module 1, head, tail]",[]
2,113,AVHR,n,5,3,5,3,Traced,False,Traced,"[10552, 10846, 145]","{'Immature module 1': {'pre': 5, 'post': 3, 'd...",8,"[Immature module 1, head, tail]","[Immature module 1, head, tail]"
3,120,AVJR,in,2,4,2,4,Traced,False,Traced,"[10552, 10846, 145]","{'Immature module 2': {'pre': 2, 'post': 4, 'd...",6,"[Immature module 2, head, tail]","[Immature module 2, head, tail]"
4,124,AVJL,in,2,6,2,6,Traced,False,Traced,"[10552, 10846, 145]","{'Immature module 2': {'pre': 2, 'post': 6, 'd...",8,"[Immature module 2, head, tail]","[Immature module 2, head, tail]"
5,177,PVQR,in,5,3,5,3,Traced,False,Traced,"[10552, 10846, 145]","{'Immature module 2': {'pre': 5, 'post': 3, 'd...",8,"[Immature module 2, head, tail]","[Immature module 2, head, tail]"
6,178,PVPR,in,7,1,7,1,Traced,False,Traced,"[10552, 10846, 145]","{'Immature module 2': {'pre': 7, 'post': 1, 'd...",8,"[Immature module 2, head, tail]","[Immature module 2, head, tail]"
7,186,PVT,in,7,1,7,1,Traced,False,Traced,"[10552, 10846, 145]","{'Immature module 2': {'pre': 7, 'post': 1, 'd...",8,"[Immature module 2, head, tail]","[Immature module 2, head, tail]"
8,187,PVPL,in,6,1,6,1,Traced,False,Traced,"[10552, 10846, 145]","{'Immature module 2': {'pre': 6, 'post': 1, 'd...",7,"[Immature module 2, head, tail]","[Immature module 2, head, tail]"
9,188,PVQL,in,7,6,7,6,Traced,False,Traced,"[10552, 10846, 145]","{'Immature module 2': {'pre': 7, 'post': 6, 'd...",13,"[Immature module 2, head, tail]","[Immature module 2, head, tail]"


In [8]:
# Fetch all downstream connections for one neuron
criteria = NeuronCriteria(instance=["PVT"])
df = queries.fetch_simple_connections(criteria)
df

,bodyId_pre,bodyId_post,weight,type_pre,type_post,instance_pre,instance_post,conn_roiInfo
0,186,19,2,in,i,PVT,RIGR,"{'Immature module 2': {'pre': 2}, 'Immature mo..."
1,186,144,2,in,n,PVT,AVKR,"{'Immature module 2': {'pre': 2}, 'Immature mo..."
2,186,104,1,in,i,PVT,AIBL,"{'Immature module 2': {'pre': 1}, 'Immature mo..."
3,186,143,1,in,n,PVT,AVKL,"{'Immature module 2': {'pre': 1}, 'Immature mo..."
4,186,149,1,in,i,PVT,RIGL,"{'Immature module 2': {'pre': 1}, 'Immature mo..."


In [10]:
# Fetch all connections between one type of neurons
criteria = NeuronCriteria(type=['i'])
neuron_df, conn_df = queries.fetch_adjacencies(criteria, criteria)
# Visualize as adjacency matrix
conn_df = merge_neuron_properties(neuron_df, conn_df, ["instance"])
connection_table_to_matrix(conn_df, 'instance', sort_by='instance')

/Users/goodehm1/miniconda3/envs/boss-general/lib/python3.11/site-packages/neuprint/utils.py:304: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  matrix = agg_weights_df.pivot(col_pre, col_post, weight_col)


instance_post,AIAL,AIAR,AIBL,AIBR,AIZL,AIZR,AVAL,AVAR,AVBL,AVBR,AVEL,AVER,RIAL,RIAR,RIBL,RIBR,RIFL,RIFR,RIR
instance_pre,,,,,,,,,,,,,,,,,,,
ADAL,0,0,0,1,0,0,1,1,3,0,0,0,0,0,0,0,0,0,0
ADAR,0,0,1,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0
AIAL,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
AIAR,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
AIBL,0,0,0,0,0,0,0,0,4,0,1,1,0,0,0,3,0,0,0
AIBR,0,0,0,0,0,0,0,0,0,1,1,1,0,0,2,0,0,0,0
AINL,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
AINR,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
AIYL,0,0,0,0,5,0,0,0,0,0,0,0,2,0,2,0,0,0,0


You can also construct a Cypher query yourself. Here's an overview of Cypher syntax: https://memgraph.com/blog/cypher-cheat-sheet 

In [21]:
# Custom cypher query returns the names of all neurons in the dataset
query = '''
        MATCH (n:Neuron) RETURN n.instance
'''
queries.fetch_custom(query)

,n.instance
0,ADAL
1,ADAR
2,ADEL
3,ADER
4,ADFL
...,...
149,RIPR
150,BWM-VL02
151,BWM-VR02
152,BWM-VL01


To see the Cypher for a query performed in the web UI, use the Show Cypher Query button in the gray bar at the top of the query window (looks like an information symbol).